### Getting Wikipedia Data for each US city

In [15]:
# Importing libraries
import json, time
import requests
import pandas as pd
from tqdm import tqdm
from dotenv import dotenv_values

In [2]:
# Code snippet sourced from Dr. David W. McDonald's "" notebook
# License: CC BY 4.0
# Original Source: 

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<harshit@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": None,
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}


In [3]:
# Code snippet sourced from Dr. David W. McDonald's "" notebook
# License: CC BY 4.0
# Original Source: 

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    
    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [4]:
# Read CSV which contains the list of cities and states in the US
df_cities = pd.read_csv("us_cities_by_state_SEPT.2023.csv")

In [5]:
# Print the first 5 rows of the dataframe
df_cities.head()

,state,page_title,url
0,Alabama,"Abbeville, Alabama","https://en.wikipedia.org/wiki/Abbeville,_Alabama"
1,Alabama,"Adamsville, Alabama","https://en.wikipedia.org/wiki/Adamsville,_Alabama"
2,Alabama,"Addison, Alabama","https://en.wikipedia.org/wiki/Addison,_Alabama"
3,Alabama,"Akron, Alabama","https://en.wikipedia.org/wiki/Akron,_Alabama"
4,Alabama,"Alabaster, Alabama","https://en.wikipedia.org/wiki/Alabaster,_Alabama"


In [6]:
# Create a list of dictionaries which will contain the pageinfo for each city
cities_data = []
total_iter = len(df_cities)
for index, row in tqdm(df_cities.iterrows(), total=total_iter):
    page_title = row['page_title'].replace(" ", "_")
    pageinfo = request_pageinfo_per_article(article_title=page_title)['query']['pages']
    cities_data.append(pageinfo)

  0%|          | 2/22157 [00:00<1:48:20,  3.41it/s]

100%|██████████| 22157/22157 [1:47:35<00:00,  3.43it/s]  


In [7]:
# Dump the data into a JSON file
json.dump(cities_data, open("cities_data.json", "w"))

### Getting Article Quality Prediction

In [117]:
# Read the JSON file
with open('cities_data.json', 'r') as file:
    data = file.read()

In [118]:
# Parse the JSON data into a list of dictionaries
data_list = json.loads(data)

# Flatten the list of dictionaries into a list of values
flattened_data = [list(item.values())[0] for item in data_list]

# Create a dataframe from the flattened data
df = pd.DataFrame(flattened_data)

In [119]:
# Print the first 5 rows of the dataframe
df.head()

,pageid,ns,title,contentmodel,pagelanguage,pagelanguagehtmlcode,pagelanguagedir,touched,lastrevid,length,talkid,fullurl,editurl,canonicalurl,watchers,redirect,new
0,104730,0,"Abbeville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1171163550,24706,281244.0,"https://en.wikipedia.org/wiki/Abbeville,_Alabama",https://en.wikipedia.org/w/index.php?title=Abb...,"https://en.wikipedia.org/wiki/Abbeville,_Alabama",NaN,NaN,NaN
1,104761,0,"Adamsville, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1177621427,18040,281272.0,"https://en.wikipedia.org/wiki/Adamsville,_Alabama",https://en.wikipedia.org/w/index.php?title=Ada...,"https://en.wikipedia.org/wiki/Adamsville,_Alabama",NaN,NaN,NaN
2,105188,0,"Addison, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1168359898,13309,281517.0,"https://en.wikipedia.org/wiki/Addison,_Alabama",https://en.wikipedia.org/w/index.php?title=Add...,"https://en.wikipedia.org/wiki/Addison,_Alabama",NaN,NaN,NaN
3,104726,0,"Akron, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1165909508,11710,281240.0,"https://en.wikipedia.org/wiki/Akron,_Alabama",https://en.wikipedia.org/w/index.php?title=Akr...,"https://en.wikipedia.org/wiki/Akron,_Alabama",NaN,NaN,NaN
4,105109,0,"Alabaster, Alabama",wikitext,en,en,ltr,2023-10-10T22:35:37Z,1179139816,20343,281444.0,"https://en.wikipedia.org/wiki/Alabaster,_Alabama",https://en.wikipedia.org/w/index.php?title=Ala...,"https://en.wikipedia.org/wiki/Alabaster,_Alabama",NaN,NaN,NaN


In [120]:
wiki_api_secrets = dotenv_values(".env")

In [121]:
# Load the username and access token
USERNAME = wiki_api_secrets['USERNAME']
ACCESS_TOKEN = wiki_api_secrets['ACCESS_TOKEN']

In [122]:
# Code snippet sourced from Dr. David W. McDonald's "wp_ores_liftwing_example" notebook
# License: CC BY 4.0
# Original Source: https://colab.research.google.com/drive/17C9xsmR9U3lJeD52UTbAedlHDetwYsxs?authuser=3#scrollTo=i7Fh0_9n3Mdd

# The current LiftWing ORES API endpoint and prediction model
API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"

# The throttling rate is a function of the Access token that you are granted when you request the token. The constants
# come from dissecting the token and getting the rate limits from the granted token. An example of that is below.
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (60.0/5000.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
#    
# Because all LiftWing API requests require some form of authentication, you need to provide your access token
# as part of the header too
REQUEST_HEADER_TEMPLATE = {
    'User-Agent': "harshit@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2023",
    'Content-Type': 'application/json',
    'Authorization': "Bearer {access_token}"
}

# This is a template for the parameters that we need to supply in the headers of an API request
REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address' : "harshit.rai.usa@gmail.com",
    'access_token'  : ACCESS_TOKEN
}

# This is a template of the data required as a payload when making a scoring request of the ORES model
ORES_REQUEST_DATA_TEMPLATE = {
    "lang":        "en",     # required that its english - we're scoring English Wikipedia revisions
    "rev_id":      "",       # this request requires a revision id
    "features":    True
}

In [130]:
# Code snippet sourced from Dr. David W. McDonald's "wp_ores_liftwing_example" notebook
# License: CC BY 4.0
# Original Source: https://colab.research.google.com/drive/17C9xsmR9U3lJeD52UTbAedlHDetwYsxs?authuser=3#scrollTo=i7Fh0_9n3Mdd

def request_ores_score_per_article(article_revid = None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT, 
                                   model_name = API_ORES_EN_QUALITY_MODEL, 
                                   request_data = ORES_REQUEST_DATA_TEMPLATE, 
                                   header_format = REQUEST_HEADER_TEMPLATE, 
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):
    
    #    Make sure we have an article revision id, email and token
    #    This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid
    
    #   Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")
    
    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)
    
    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free data
        # source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
        prediction = json_response['enwiki']['scores'][str(article_revid)]['articlequality']['score']['prediction']
        prediction_score = json_response['enwiki']['scores'][str(article_revid)]['articlequality']['score']['probability'][prediction]
        if type(prediction_score) is not float:
            print("Prediction score is not a float")
    except Exception as e:
        print(e)
        prediction, prediction_score = (None, None)
    return (prediction, prediction_score)


In [ ]:
# Create two new columns in the dataframe to store the article quality and quality score
total_iter = len(df)
for index, row in tqdm(df.iterrows(), total=total_iter):
    article_revid = row['lastrevid']
    prediction, prediction_score = request_ores_score_per_article(article_revid = article_revid)
    df.loc[index, 'article_quality'] = prediction
    df.loc[index, 'quality_score'] = prediction_score

In [150]:
# Filter the dataframe to only include articles with a quality score
df_failed = df[df['quality_score'].isna()]
print(f"Number of failed requests: {len(df_failed)}")

Number of failed requests: 4220


In [149]:
# Rerun the ORES API call for the articles that failed
total_iter = len(df_failed)
for index, row in tqdm(df_failed.iterrows(), total=total_iter):
    article_revid = row['lastrevid']
    prediction, prediction_score = request_ores_score_per_article(article_revid = article_revid)
    df.loc[index, 'article_quality'] = prediction
    df.loc[index, 'quality_score'] = prediction_score

100%|██████████| 4220/4220 [45:38<00:00,  1.54it/s]  


In [151]:
# Confirm that there are no more failed requests
df_failed = df[df['quality_score'].isna()]
print(f"Number of failed requests: {len(df_failed)}")

Number of failed requests: 0


In [152]:
# Save the dataframe to a CSV file
df.to_csv("cities_data_with_quality.csv", index=False)

In [154]:
df['article_quality'].value_counts()

article_quality
C        13343
GA        4809
Start     2198
B          914
Stub       679
FA         214
Name: count, dtype: int64

In [246]:
# Select only the columns that are required for the analysis
df = df[['title', 'lastrevid', 'article_quality']]
df.columns = ['article_title', 'revision_id', 'article_quality']

In [247]:
# Derive state name by joining on the page title found in the original dataset
df = df.merge(df_cities[['page_title', 'state']], left_on='article_title', right_on='page_title', how='left')
# Drop the page title column
df = df.drop(columns=['page_title'])

In [248]:
# Replace _ with space in the state column
df['state'] = df['state'].str.replace("_", " ")

# Replace 'Georgia (U.S. state)' with 'Georgia' to match the state name in the US States by Region dataset
df.loc[df['state'] == 'Georgia (U.S. state)', 'state'] = 'Georgia'

In [249]:
df.head()

,article_title,revision_id,article_quality,state
0,"Abbeville, Alabama",1171163550,C,Alabama
1,"Abbeville, Alabama",1171163550,C,Alabama
2,"Adamsville, Alabama",1177621427,C,Alabama
3,"Adamsville, Alabama",1177621427,C,Alabama
4,"Addison, Alabama",1168359898,C,Alabama


### Combining the Datasets

In [250]:
# Read XLSX file containing the list of states and their regions
df_states = pd.read_excel("US States by Region - US Census Bureau.xlsx")

In [251]:
df_states.head(10)

,REGION,DIVISION,STATE
0,Northeast,NaN,NaN
1,NaN,New England,NaN
2,NaN,NaN,Connecticut
3,NaN,NaN,Maine
4,NaN,NaN,Massachusetts
5,NaN,NaN,New Hampshire
6,NaN,NaN,Rhode Island
7,NaN,NaN,Vermont
8,NaN,Middle Atlantic,NaN
9,NaN,NaN,New Jersey


In [252]:
# Use forward fill to fill missing values
df_states['REGION'].fillna(method='ffill', inplace=True)
df_states['DIVISION'].fillna(method='ffill', inplace=True)
df_states['STATE'].fillna(method='ffill', inplace=True)

# Filter out rows where the STATE is not null
df_states = df_states[df_states['STATE'].notnull()]

# Reset the index
df.reset_index(drop=True, inplace=True)

In [253]:
df_states.head(10)

,REGION,DIVISION,STATE
2,Northeast,New England,Connecticut
3,Northeast,New England,Maine
4,Northeast,New England,Massachusetts
5,Northeast,New England,New Hampshire
6,Northeast,New England,Rhode Island
7,Northeast,New England,Vermont
8,Northeast,Middle Atlantic,Vermont
9,Northeast,Middle Atlantic,New Jersey
10,Northeast,Middle Atlantic,New York
11,Northeast,Middle Atlantic,Pennsylvania


In [254]:
# Merge the two dataframes on the state name to get the region and division
df = df.merge(df_states[['DIVISION','STATE']], left_on='state', right_on='STATE', how='left')

# Drop the STATE column
df = df.drop(columns=['STATE'])

In [255]:
df.head()

,article_title,revision_id,article_quality,state,DIVISION
0,"Abbeville, Alabama",1171163550,C,Alabama,East South Central
1,"Abbeville, Alabama",1171163550,C,Alabama,East South Central
2,"Adamsville, Alabama",1177621427,C,Alabama,East South Central
3,"Adamsville, Alabama",1177621427,C,Alabama,East South Central
4,"Addison, Alabama",1168359898,C,Alabama,East South Central


In [256]:
# Check for missing values
df.isna().sum()

article_title      0
revision_id        0
article_quality    0
state              0
DIVISION           0
dtype: int64

In [278]:
# Read population data from the XLSX file from the 10th row with given headers
df_population = pd.read_excel("NST-EST2022-POP.xlsx", header=None, skiprows=9, names=['State','Base 2020 Population', '2020 Estimate', '2021 Estimate', '2022 Estimate'],
                              skipfooter=5)

# Drop null values
df_population = df_population[df_population['State'].notnull()]

In [280]:
# Remove the "." from the state names
df_population['State'] = df_population['State'].str.replace(".", "")

In [281]:
df_population.head()

,State,Base 2020 Population,2020 Estimate,2021 Estimate,2022 Estimate
0,Alabama,5024356.0,5031362.0,5049846.0,5074296.0
1,Alaska,733378.0,732923.0,734182.0,733583.0
2,Arizona,7151507.0,7179943.0,7264877.0,7359197.0
3,Arkansas,3011555.0,3014195.0,3028122.0,3045637.0
4,California,39538245.0,39501653.0,39142991.0,39029342.0


In [282]:
# Keep only state name and 2022 estimate columns
df_population = df_population[['State', '2022 Estimate']]

In [283]:
# Merge df with df_population on the state name
df = df.merge(df_population, left_on='state', right_on='State', how='left')

In [284]:
df.head()

,article_title,revision_id,article_quality,state,DIVISION,State,2022 Estimate
0,"Abbeville, Alabama",1171163550,C,Alabama,East South Central,Alabama,5074296.0
1,"Abbeville, Alabama",1171163550,C,Alabama,East South Central,Alabama,5074296.0
2,"Adamsville, Alabama",1177621427,C,Alabama,East South Central,Alabama,5074296.0
3,"Adamsville, Alabama",1177621427,C,Alabama,East South Central,Alabama,5074296.0
4,"Addison, Alabama",1168359898,C,Alabama,East South Central,Alabama,5074296.0


In [285]:
# Check for missing values in df
df.isna().sum()

article_title      0
revision_id        0
article_quality    0
state              0
DIVISION           0
State              0
2022 Estimate      0
dtype: int64

In [287]:
# Drop article title duplicates 
df = df.drop_duplicates(subset=['article_title'])

In [289]:
# Remove all articles with article name in list. This list represents articles that are not cities
articles_to_remove = ['Federal Information Processing Standards',
 'American National Standards Institute',
 'Geographic Names Information System',
 'Wikipedia:Citation needed',
 '2020 United States census',
 '2010 United States census',
 'County (United States)',
 'Population',
 'Square mile']
for index, row in df.iterrows():
    if row['article_title'] in articles_to_remove:
        df.drop(index, inplace=True)
        print(f"Dropped {row['article_title']}")

Dropped 2020 United States census
Dropped 2010 United States census
Dropped County (United States)
Dropped Population
Dropped Square mile
Dropped Federal Information Processing Standards
Dropped American National Standards Institute
Dropped Geographic Names Information System
Dropped Wikipedia:Citation needed


In [291]:
# Select only the columns that are required for the analysis
df = df[['state', 'DIVISION', '2022 Estimate', 'article_title', 'revision_id', 'article_quality']]

In [292]:
# Rename the columns
df.columns = ['state', 'division', 'population', 'article_title', 'revision_id', 'article_quality']

In [293]:
# Write the dataframe to a CSV file
df.to_csv("wp_scored_city_articles_by_state.csv", index=False)